In [3]:
%pip install babel

Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import os
from pathlib import Path
from babel.numbers import format_currency

BASE_DIR = Path.cwd().parent

DATA_RAW = BASE_DIR/'data'/'raw'

lista_dfs = []

for caminho in DATA_RAW.glob("*.csv"):

    df = pd.read_csv(
        caminho,
        encoding='latin1',
        sep=';',
        decimal=',',
        skiprows=1  
    )
    
    lista_dfs.append(df)

datas = pd.concat(lista_dfs, ignore_index=True)


datas Brutos importados e características gerais

In [5]:
datas

,ANO,MES,SENADOR,TIPO_DESPESA,CNPJ_CPF,FORNECEDOR,DOCUMENTO,DATA,DETALHAMENTO,VALOR_REEMBOLSADO,COD_DOCUMENTO
0,2018,1,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",004.948.028-63,GILBERTO PISELO DO NASCIMENTO,NaN,02/01/2018,Despesa com pagamento de aluguel de imóvel par...,6000.00,2088462
1,2018,1,ACIR GURGACZ,"Passagens aéreas, aquáticas e terrestres nacio...",63.764.211/0001-10,TRANSCONTINENTAL AGÊNCIA DE VIAGENS LTDA,M2CJMM,02/01/2018,"Companhia Aérea: AVIANCA, Localizador: M2CJMM....",274.99,2088476
2,2018,1,ACIR GURGACZ,"Passagens aéreas, aquáticas e terrestres nacio...",63.764.211/0001-10,TRANSCONTINENTAL AGÊNCIA DE VIAGENS LTDA,SFEXYF,02/01/2018,"Companhia Aérea: LATAM, Localizador: SFEXYF. P...",1094.29,2088528
3,2018,1,ACIR GURGACZ,"Passagens aéreas, aquáticas e terrestres nacio...",63.764.211/0001-10,TRANSCONTINENTAL AGÊNCIA DE VIAGENS LTDA,V64HTR,24/01/2018,"Companhia Aérea: AZUL, Localizador: V64HTR. Pa...",1553.99,2088475
4,2018,2,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",004.948.028-63,GILBERTO PISELO DO NASCIMENTO,002/18,01/02/2018,Despesa com pagamento de aluguel de imóvel par...,6000.00,2088477
...,...,...,...,...,...,...,...,...,...,...,...
93843,2022,12,ZEQUINHA MARINHO,"Passagens aéreas, aquáticas e terrestres nacio...",22.052.777/0001-32,Exceller Tour,WIXHAI,06/12/2022,"Companhia Aérea: LATAM, Localizador: WIXHAI. P...",2893.04,2191398
93844,2022,12,ZEQUINHA MARINHO,"Passagens aéreas, aquáticas e terrestres nacio...",22.052.777/0001-32,Exceller Tour,WITOLM,09/12/2022,"Companhia Aérea: GOL, Localizador: WITOLM. Pas...",1180.19,2192272
93845,2022,12,ZEQUINHA MARINHO,"Passagens aéreas, aquáticas e terrestres nacio...",22.052.777/0001-32,Exceller Tour,THPKVQ,20/12/2022,"Companhia Aérea: TAM, Localizador: THPKVQ. Pas...",2671.90,2192274
93846,2022,12,ZEQUINHA MARINHO,"Passagens aéreas, aquáticas e terrestres nacio...",22.052.777/0001-32,Exceller Tour,QNN9HX,21/12/2022,"Companhia Aérea: AZUL, Localizador: QNN9HX. Pa...",1334.31,2192244


In [6]:
datas.shape


(93848, 11)

Verificação de valores nulos

In [7]:
datas.isnull().sum()

ANO                      0
MES                      0
SENADOR                  0
TIPO_DESPESA             0
CNPJ_CPF                 0
FORNECEDOR               0
DOCUMENTO             3979
DATA                     0
DETALHAMENTO         36383
VALOR_REEMBOLSADO        0
COD_DOCUMENTO            0
dtype: int64

Retirando dados nulos ou inconclusivos da base de dados

In [8]:
datas = datas[~datas['DOCUMENTO'].isnull()]

In [9]:
datas = datas[~datas['DETALHAMENTO'].isnull()]

Verificando arquivos

In [ ]:
datas.isnull().sum()

In [11]:
datas.shape

(54798, 11)

Tratamento de dados gerais

1 - Garantindo que a coluna de reembolso seja lida como número, em caso de estar em texto.

In [12]:
datas['VALOR_REEMBOLSADO'] = pd.to_numeric(datas['VALOR_REEMBOLSADO'], errors= 'coerce')

2 - Retirada de caracteres especiais do CNPJ e CPF dos fornecedores

In [13]:
datas['CNPJ_CPF'] = datas['CNPJ_CPF'].astype(str).replace(r'\D', '', regex = True)

3 - Criando uma coluna de competência para aspectos contábeis

In [14]:
datas['COMPETENCIA'] = pd.to_datetime(datas['ANO'].astype(str) + '/' + datas['MES'].astype(str), format = '%Y/%m')


In [15]:
datas['COMPETENCIA'] = datas['COMPETENCIA'].dt.to_period('M')
#Garantindo que a competência seja do tipo m/ano e não o padrão dia/mes/ano

4 - Formatando o texto seja totalmente em maiúsculo para facilitar a compreensão

In [16]:
datas['TIPO_DESPESA']  = datas['TIPO_DESPESA'].str.strip().str.upper()

In [17]:
datas['DETALHAMENTO'] = datas['DETALHAMENTO'].str.strip().str.upper()

5 - Garantindo que o número seja reconhecido como número e não como string.

In [18]:
datas['COD_DOCUMENTO'] = pd.to_numeric(datas['COD_DOCUMENTO'], errors='coerce')

In [25]:
DATAS_PROCESSED = BASE_DIR / 'data' / 'processed'

datas.to_csv(DATAS_PROCESSED/'ceaps_tratados18-22.csv', sep= ';', decimal= '.', encoding= 'latin1', index= False)